In [ ]:
%matplotlib inline
import sys
sys.path.append("..")

# DeepTrack 2.0 - Tracking multiple particles with a U-net

This tutorial demonstrates how to track multiple particles using a U-net with DeepTrack 2.0.

The U-net receives as input an image that may or may not contain particles and outputs an image whose pixels represent the probability that there is a particle nearby. Specifically, each pixel has a value between 0 (high confidence that there is no particle close by) and 1 (high confidence that there is a nearby particle).

This tutorial should be perused after the tutorials [deeptrack_introduction_tutorial](deeptrack_introduction_tutorial.ipynb) and [tracking_particle_cnn_tutorial](tracking_particle_cnn_tutorial.ipynb).

## 1. Setup

Imports needed for this tutorial.

In [ ]:
from deeptrack.scatterers import PointParticle
from deeptrack.optics import Fluorescence
from deeptrack.noises import Poisson, Offset
from deeptrack.generators import Generator
from deeptrack.models import unet
from deeptrack.losses import weighted_crossentropy, sigmoid, flatten

import numpy as np
import matplotlib.pyplot as plt

## 2. Define the particle

For this example, we consider point particles (point light scatterers). A point particle is an instance of the class `PointParticle`, defined by its intensity and its position. Here, the position is randomized using a lambda function. More details can be found in the tutorial [tracking_particle_cnn_tutorial](tracking_particle_cnn_tutorial.ipynb).

In [ ]:
particle = PointParticle(                                         
    intensity=100,
    position=lambda: np.random.rand(2) * 256, 
    position_unit="pixel"
)

## 3. Define the optical system 

Next, we need to define the properties of the optical system. This is done using an instance of the class `Fluorescence`, which takes a set of light scatterers (particles) and convolves them with the pupil function (point spread function) of the optical system. More details can be found in the tutorial [tracking_particle_cnn_tutorial](tracking_particle_cnn_tutorial.ipynb).

In [ ]:
fluorescence_microscope = Fluorescence(
    NA=0.7,                
    resolution=1e-6,     
    magnification=10,
    wavelength=680e-9,
    output_region=(0, 0, 256, 256)
)

## 4. Define noises

We introduce two sources of noise (see also [noises_example](../examples/noises_example.ipynb)):
1. A background random offset between 0 and 1.
2. A Poisson noise with a random SNR between 20 and 50.

In [ ]:
offset = Offset(
    offset=lambda: np.random.rand()*1
)

poisson_noise = Poisson(
    snr=lambda: np.random.rand()*30 + 20
)

## 5. Define the image features

We want images with a random number of particles between 1 and 10, a background offset, and Poisson noise.

In [ ]:
num_particles = lambda: np.random.randint(1, 11)

image_features = fluorescence_microscope(particle**num_particles) + offset + poisson_noise

## 6. Plot example images

Now, we visualize some example images. At each iteration, we call the method `.update()` to refresh the random features in the image (particle number, particle positions, offset level, and Poisson noise). Afterwards we call the method `.plot()` to generate and display the image.

In [ ]:
for i in range(4):
    image_features.update()
    output_image = image_features.plot(cmap="gray")

## 7. Create the target images

We define a function that uses the generated images to create the target images to be used in the training. Here the target image is binary image, where each pixel is `1` if it is within `CIRCLE_RADIUS` distance from any particle in the input image, and 0 otherwise. 

In [2]:
# Creates an image with circles of radius two at the same position 
# as the particles in the input image.

CIRCLE_RADIUS = 3

def get_target_image(image_of_particles):
    target_image = np.zeros(image_of_particles.shape)
    X, Y = np.meshgrid(
        np.arange(0, image_of_particles.shape[0]), 
        np.arange(0, image_of_particles.shape[1])
    )

    for property in image_of_particles.properties:
        if "position" in property:
            position = property["position"]

            distance_map = (X - position[1])**2 + (Y - position[0])**2
            target_image[distance_map < CIRCLE_RADIUS**2] = 1
    
    return target_image

Here, we show images and targets side by side.

In [3]:
for i in range(4):
    image_features.update()
    image_of_particles = image_features.resolve()

    target_image = get_target_image(image_of_particles)

    plt.subplot(1,2,1)
    plt.imshow(np.squeeze(image_of_particles), cmap="gray")
    plt.title("Input Image")
    
    plt.subplot(1,2,2)
    plt.imshow(np.squeeze(target_image), cmap="gray")
    plt.title("Target image")
    
    plt.show()

NameError: name 'image_features' is not defined

## 8. Define image generator

We define a generator that creates images and targets in batches of 8.

In [ ]:
generator = Generator().generate(
    image_features, 
    get_target_image,
    batch_size=8
)

## 9. Define the neural network model

The neural network architecture used is a U-Net, which is a fully convolutional model used for image-to-image transformations. We create this model by calling the function `unet` (see also [models_example](../examples/models_example.ipynb)).

Since the desired output is a binary image, we will be using crossentropy as loss. Furthermore, since the target image is disproportionaly populated by 0s (any pixel is much more likely to be a zero than a one), we weight the loss such that false negatives are penalized ten times more than the false positives. 

The model can be customized by passing the following arguments:

* `input_shape`: Size of the images to be analyzed. The first two values can be set to `None` to allow arbitrary sizes.

* `conv_layers_dimensions`: Number of convolutions in each convolutional layer during down-
    and upsampling.
    
* `base_conv_layers_dimensions`: Number of convolutions in each convolutional layer at the base
    of the unet, where the image is the most downsampled.

* `output_conv_layers_dimensions`: Number of convolutions in each convolutional layer after the
    upsampling.
    
* `steps_per_pooling`: Number of convolutional layers between each pooling and upsampling
    step.

* `number_of_outputs`: Number of convolutions in output layer.

* `output_activation`: The activation function of the output.

* `loss`: The loss function of the network.

* `optimizer`: The the optimizer used for training.

* `metrics`: Additional metrics to evaulate during training.

In [ ]:
model = unet(
    (256, 256, 1), 
    conv_layers_dimensions=[8, 16, 32],
    base_conv_layers_dimensions=[32, 32], 
    loss=flatten(weighted_crossentropy((10, 1)))
)

model.summary()

## 10. Train the model

The model is trained by calling `.fit()`. This will take some time on the order of 10s of minutes.

In [ ]:
model.fit(
    generator, 
    epochs=50,          
    steps_per_epoch=20
)

## 11. Visualize the model performance

Finally, we evaluate the model performance by showing the model output besides the input image and the target image.

In [ ]:
input_image, target_image = next(generator)

for i in range(input_image.shape[0]):
    
    predicted_image = model.predict(input_image)
    
    plt.subplot(1,3,1)
    plt.imshow(np.squeeze(input_image[i, :, :, 0]), cmap="gray")
    plt.title("Input Image")

    plt.subplot(1,3,2)
    plt.imshow(np.squeeze(predicted_image[i, :, :, 0]), cmap="gray")
    plt.title("Predicted Image")
    
    plt.subplot(1,3,3)
    plt.imshow(np.squeeze(target_image[i, :, :, 0] > 0.5), cmap="gray")
    plt.title("Target image")

    plt.show()